In [ ]:
import logging

# disable logging
logging.disable(logging.CRITICAL)
from agent.dqn import DQNAgent

agent = DQNAgent(
    default_root_dir="./training-results/TRASH",
    num_iterations=200,
    replay_buffer_size=150,
    warm_start=100,
    capacity={"episodic": 4, "semantic": 4, "short": 10}
)
# agent.fill_replay_buffer()
agent.train()

In [3]:
from agent.dqn.utils import MultiAgentReplayBuffer

In [111]:
from agent.dqn.utils import MultiAgentReplayBuffer
import numpy as np
import torch
device = "cpu"

buffer_combined = MultiAgentReplayBuffer(
    agent.replay_buffer_mm, agent.replay_buffer_explore
)
batch_mm, batch_explore = buffer_combined.sample_batch(sample_same_index=True)

assert np.array_equal(
    batch_mm["rews"], batch_explore["rews"]
), "Rewards are not the same."
assert np.array_equal(
    batch_mm["done"], batch_explore["done"]
), "Dones are not the same."

s_mm = batch_mm["obs"]
s_next_mm = batch_mm["next_obs"]
a_mm = batch_mm["acts"]
r_mm = torch.FloatTensor(batch_mm["rews"].reshape(-1, 1)).to(device)
d_mm = torch.FloatTensor(batch_mm["done"].reshape(-1, 1)).to(device)

s_explore = batch_explore["obs"]
s_next_explore = batch_explore["next_obs"]
a_explore = batch_explore["acts"]
r_explore = torch.FloatTensor(batch_explore["rews"].reshape(-1, 1)).to(device)
d_explore = torch.FloatTensor(batch_explore["done"].reshape(-1, 1)).to(device)

In [108]:
# Flatten the array of lists into a single list of integers
a_explore = [item[0] for item in a_explore]  # Extract the single element from each sublist

# # Convert the list to a PyTorch tensor
# tensor = torch.tensor(flattened_list)

# # Reshape the tensor to have the shape (32, 1)
# reshaped_tensor = tensor.view(32, 1)

# # Print the reshaped tensor
# # print(reshaped_tensor)

In [117]:
torch.LongTensor([item[0] for item in a_explore]).reshape(-1, 1).to(device)

tensor([[3],
        [1],
        [1],
        [3],
        [4],
        [4],
        [1],
        [2],
        [0],
        [0],
        [2],
        [1],
        [0],
        [0],
        [3],
        [2],
        [2],
        [4],
        [4],
        [3],
        [4],
        [1],
        [4],
        [0],
        [0],
        [0],
        [2],
        [0],
        [3],
        [1],
        [0],
        [2]])

In [103]:
# convert it to tensor
a_explore

array([list([4]), list([4]), list([0]), list([3]), list([0]), list([3]),
       list([1]), list([2]), list([3]), list([2]), list([4]), list([0]),
       list([4]), list([2]), list([1]), list([2]), list([4]), list([4]),
       list([2]), list([0]), list([3]), list([2]), list([0]), list([0]),
       list([1]), list([4]), list([2]), list([2]), list([3]), list([4]),
       list([0]), list([4])], dtype=object)

In [101]:
torch.LongTensor(a_explore.reshape(-1, 1)).to(device)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [71]:
import torch

# Create a tensor
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Pad indices to the maximum length, in this case 3, using a padding value that is invalid (-1 or could use max column index if -1 is problematic)
padded_indices = torch.tensor([
    [0, -1, -1],
    [1, 2, -1],
    [0, 1, 2]
])

In [72]:
# Use torch.gather to gather elements from the tensor based on the padded index
# Mask to ignore padded values (-1 index will cause an error, thus handle via valid range or other methods)
valid_mask = padded_indices >= 0
valid_mask

tensor([[ True, False, False],
        [ True,  True, False],
        [ True,  True,  True]])

In [73]:
padded_indices[~valid_mask] = 0  # Temporarily replace invalid indices with 0 to avoid runtime error
padded_indices

tensor([[0, 0, 0],
        [1, 2, 0],
        [0, 1, 2]])

In [74]:
gathered = torch.gather(tensor, dim=1, index=padded_indices)
gathered

tensor([[1, 1, 1],
        [5, 6, 4],
        [7, 8, 9]])

In [77]:
gathered[~valid_mask] = -1  # Replace output from invalid indices with -1 or another flag value
gathered

tensor([[ 1, -1, -1],
        [ 5,  6, -1],
        [ 7,  8,  9]])

In [92]:
torch.tensor([[1, 2], [2, -1]]).max(dim=1, keepdim=True)[0]

tensor([[2],
        [2]])

In [123]:
torch.tensor([1]).detach().cpu().numpy().item()

1

In [29]:
import torch

# Create a tensor
tensor = torch.tensor([[1, 2, 3],
                       [4, 5, 6],
                       [7, 8, 9]])

# Create an index tensor
index = torch.tensor([[0, 2],
                      [1, 0],
                      [2, 1]])

# Use torch.gather to gather elements from the tensor based on the index
gathered = torch.gather(tensor, dim=1, index=index)

print(gathered)


tensor([[1, 3],
        [5, 4],
        [9, 8]])


In [59]:
index = np.array([{}]*3)
index[0] = [0]
index[1] = [1,2]
index[2] = [0,1,2]
index

array([list([0]), list([1, 2]), list([0, 1, 2])], dtype=object)

In [60]:
torch.tensor(index)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

In [1]:
import logging

# disable logging
logging.disable(logging.CRITICAL)
from agent.dqn import DQNAgent
import gymnasium as gym
import random
from humemai.policy import manage_memory, encode_all_observations

agent = DQNAgent(
    default_root_dir="./training-results/TRASH",
    num_iterations=100,
    replay_buffer_size=100,
    warm_start=100,
)

env = gym.make("room_env:RoomEnv-v2", room_size="l")
observations, info = env.reset()
rewards = 0

# while True:
#     observations, reward, done, truncated, info = env.step(
#         (
#             ["random answer"] * len(observations["questions"]),
#             random.choice(["north", "east", "south", "west", "stay"]),
#         )
#     )
#     rewards += reward
#     if done or truncated:
#         break

agent.fill_replay_buffer()

Running on cpu


/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:168: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed a `seed` instead of using `Env.seed` for resetting the environment random number generator.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/gymnasium/utils/passive_env_checker.py:181: DeprecationWarning: WARN: Current gymnasium version requires that `Env.reset` can be passed `options` to allow the environment initialisation to be passed additional information.
  logger.deprecation(
/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/gymnasiu

In [24]:
agent.replay_buffer_mm.sample_batch()["acts"][0]

[0, 1, 0, 2, 2]

In [3]:
agent.memory_systems

{   'episodic': {   '_frozen': False,
    'capacity': 16,
    'entries': [   ],
    'remove_duplicates': False,
    'type': 'episodic'},
    'semantic': {   '_frozen': False,
    'capacity': 16,
    'entries': [   ],
    'type': 'semantic'},
    'short': {   '_frozen': False,
    'capacity': 10,
    'entries': [   [   'room_000',
                       'north',
                       'wall',
                       0],
                   [   'room_000',
                       'east',
                       'room_001',
                       0],
                   [   'room_000',
                       'south',
                       'wall',
                       0],
                   [   'room_000',
                       'west',
                       'wall',
                       0],
                   [   'agent',
                       'atlocation',
                       'room_000',
                       0]],
    'type': 'short'}}

In [4]:
for a_mm_, mem_short in zip([0, 1, 2, 0, 1], agent.memory_systems.short):
    print(mem_short)
    manage_memory(agent.memory_systems, agent.action_mm2str[a_mm_], mem_short)


2024-05-03 11:30:46.669 INFO memory - add: memory entry ['room_000', 'north', 'wall', 0] added. Now there are in total of 1 memories!
2024-05-03 11:30:46.672 INFO memory - forget: ['room_000', 'north', 'wall', 0] forgotten!
2024-05-03 11:30:46.673 INFO memory - add: memory entry ['room_000', 'east', 'room_001', 1] added. Now there are in total of 1 memories!
2024-05-03 11:30:46.674 INFO memory - forget: ['room_000', 'east', 'room_001', 0] forgotten!
2024-05-03 11:30:46.675 INFO memory - forget: ['room_000', 'south', 'wall', 0] forgotten!
2024-05-03 11:30:46.676 INFO memory - add: memory entry ['room_000', 'west', 'wall', 0] added. Now there are in total of 2 memories!
2024-05-03 11:30:46.676 INFO memory - forget: ['room_000', 'west', 'wall', 0] forgotten!
2024-05-03 11:30:46.677 INFO memory - add: memory entry ['agent', 'atlocation', 'room_000', 1] added. Now there are in total of 2 memories!
2024-05-03 11:30:46.678 INFO memory - forget: ['agent', 'atlocation', 'room_000', 0] forgotten

['room_000', 'north', 'wall', 0]
['room_000', 'east', 'room_001', 0]
['room_000', 'south', 'wall', 0]
['room_000', 'west', 'wall', 0]
['agent', 'atlocation', 'room_000', 0]


In [6]:
agent.memory_systems

{   'episodic': {   '_frozen': False,
    'capacity': 16,
    'entries': [   [   'room_000',
                       'north',
                       'wall',
                       0],
                   [   'room_000',
                       'west',
                       'wall',
                       0]],
    'remove_duplicates': False,
    'type': 'episodic'},
    'semantic': {   '_frozen': False,
    'capacity': 16,
    'entries': [   [   'room_000',
                       'east',
                       'room_001',
                       1],
                   [   'agent',
                       'atlocation',
                       'room_000',
                       1]],
    'type': 'semantic'},
    'short': {   '_frozen': False,
    'capacity': 10,
    'entries': [   ],
    'type': 'short'}}

In [1]:
from agent.dqn.utils import ReplayBuffer
import random

buffer = ReplayBuffer(8, 4)

for _ in range(6):

    rand_dict_1 = {str(i): str(random.randint(0, 10)) for i in range(3)}
    rand_dict_2 = {str(i): str(random.randint(0, 10)) for i in range(3)}
    action = [i for i in range(random.randint(1, 10))]
    buffer.store(
        *[
            rand_dict_1,
            action,
            random.choice([-1, 1]),
            rand_dict_2,
            random.choice([False, True]),
        ]
    )

/home/tk/.virtualenvs/agent-room-env-v2-gnn/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
sample = buffer.sample_batch()
sample

{'obs': array([{'0': '3', '1': '4', '2': '5'}, {'0': '5', '1': '1', '2': '0'},
        {'0': '9', '1': '6', '2': '2'}, {'0': '7', '1': '2', '2': '4'}],
       dtype=object),
 'next_obs': array([{'0': '5', '1': '0', '2': '5'}, {'0': '5', '1': '1', '2': '9'},
        {'0': '5', '1': '8', '2': '4'}, {'0': '6', '1': '9', '2': '4'}],
       dtype=object),
 'acts': array([list([0, 1, 2, 3, 4, 5]), list([0, 1, 2, 3, 4, 5, 6, 7]),
        list([0, 1]), list([0])], dtype=object),
 'rews': array([ 1., -1., -1.,  1.], dtype=float32),
 'done': array([0., 1., 1., 0.], dtype=float32)}

In [8]:
sample["acts"]

array([list([0, 1, 2, 3, 4, 5, 6]), list([0, 1, 2]),
       list([0, 1, 2, 3, 4, 5, 6, 7]), list([0, 1, 2, 3])], dtype=object)

In [10]:
sample["acts"].shape

(4,)

In [15]:
sample["acts"][3]

[0, 1, 2, 3]